### Mask R-CNN on Unreal Dataset

This notebook is used to demonstrate the applicaiton of Mask R-CNN to simulation images from the unreal engine.

In [1]:
import os, argparse
import numpy as np
import model as modellib # Mask RCNN
from tqdm import tqdm
from unreal_config  import UnrealConfig
from unreal_dataset import UnrealDataset
from unreal_utils   import MODEL_DIR, limit_GPU_usage, compute_mean_AP
from pathlib import Path

%matplotlib inline 

/home/naoki/miniconda3/envs/mrcnn_unreal/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
HOME_DIR = str(Path.home())

OBJECT_DESC_PATH = '{}/datasets/unreal/unreal dataset 2/objects_description.json'.format(HOME_DIR)
IMAGES_PATH      = '{}/datasets/unreal/unreal dataset 2/images'.format(HOME_DIR)

WEIGHT_PATH='unreal_model_weights.h5'

limit_GPU_usage()

class InferenceConfig(UnrealConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1  
config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(WEIGHT_PATH, by_name=True)

In [3]:
def generate_bbox2d(object_desc_path, images_path, data_dir):
    dataset = UnrealDataset()
    dataset.populate(object_desc_path, images_path)
    dataset.prepare()
    
    class_names = ['', 'people', 'car']
    for image_id in tqdm(dataset.image_ids):
        # Load image and ground truth data
        result = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
        if len(result)==5:
            image, image_meta, class_ids, gt_bbox, gt_mask = result
        else:
            image, image_meta, gt_bbox, gt_mask = result
            class_ids = gt_bbox[:,4]
            gt_bbox = gt_bbox[:, :4]
        molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        npz_file = '{}{}{}'.format(
            data_dir, os.sep, dataset.image_reference(image_id).split(os.sep)[-1].replace('.png', '.npz'))        
        np.savez_compressed(npz_file, 
                            rois=r['rois'], 
                            class_ids=r['class_ids'], 
                            class_names=class_names, # TODO not very efficient
                            scores=r['scores'], 
                            masks=r['masks'])    

In [4]:
generate_bbox2d(OBJECT_DESC_PATH, IMAGES_PATH, 'data')

100%|██████████| 500/500 [08:40<00:00,  1.04s/it]
